In [ ]:
import os
import warnings
warnings.filterwarnings(action='ignore')

from __future__ import print_function
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

# PyTorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader


In [1]:
import numpy as np
import pandas as pd
import h5py
import torch
from torch.utils.data import Dataset
import math

def load_dataset(filename):
    with h5py.File(filename, 'r') as f:
        if 'data' in f:
            data = f['data']
            if isinstance(data, h5py.Dataset):
                print("Type du dataset 'data':", type(data))
                print("Forme du dataset 'data':", data.shape)
                data_matrix = data[:]  
                print("Matrice de données:", data_matrix)
                return data_matrix
            elif isinstance(data, h5py.Group):
                data_list = []
                trace_names = []
                for key in data.keys():
                    dataset = data[key]
                    if isinstance(dataset, h5py.Dataset):
                        # Vérifier la forme des données extraites
                        if dataset.ndim == 2:
                            data_array = np.zeros((1, 3, dataset.shape[1]), dtype=np.float32)  # Créer une matrice vide de quatre dimensions
                            for i in range(3):  # Boucle à travers les canaux
                                data_array[:, i, :] = dataset[i, :].reshape(1,  dataset.shape[1])  # Réorganiser les données dans le bon ordre
                            data_list.append(data_array)
                            #trace_names.append(key)
                            # Concaténer les noms de clés avec un tableau de zéros
                            trace_names = np.concatenate([np.array([key] ).reshape(-1, 1) for key in data.keys()])
                            # Remodeler en un vecteur colonne
                            trace_names=trace_names.reshape(-1, 1)                          
                            #trace_names['trace_names']=trace_names
                        
                        else:
                            print(f"Les données pour la clé '{key}' ne sont pas sous forme de tableau 2D.")
                if data_list:
                    stacked_data_matrix = np.vstack(data_list)  # Empiler les matrices de données dans une seule matrice
                    return stacked_data_matrix, trace_names
                    
# Utilisation de la foncti, our charger et convertir le dataset à partir du fichier HDF5


filename1 = '20kEarthquakeAugmente.hdf5'
noise_matrix, trace_names_noise = load_dataset(filename1)

filename2 = '20kNoiseAugmente.hdf5'
events_matrix, trace_names_events = load_dataset(filename2)

print (events_matrix.shape)

(20000, 3, 12000)


In [2]:
# Création d'une colonne 'source_type' avec des valeurs constantes de 1
source_type_n = np.zeros((len(trace_names_noise), 1), dtype=int)

# Création d'une colonne 'source_type' avec des valeurs constantes de 1
source_type_e = np.ones((len(trace_names_events), 1), dtype=int)

# Concaténation de la colonne 'source_type' avec trace_names_noise
trace_names_n = np.concatenate((trace_names_noise, source_type_n), axis=1)
trace_names_e = np.concatenate((trace_names_events, source_type_e), axis=1)


trace_names_e = np.concatenate((trace_names_events[:len(trace_names_noise)], source_type_e[:len(trace_names_noise)]), axis=1)
events_matrix= events_matrix[:len(noise_matrix),:,:]
print(events_matrix.shape)
print(noise_matrix.shape)

# Vérification de la forme de la matrice résultante
print(trace_names_e)

(20000, 3, 12000)
(20000, 3, 12000)
[['20050103T055721.MN.CUC..HH' '1']
 ['20050103T194108.IV.BRMO..EH' '1']
 ['20050103T194108.IV.CTI..EH' '1']
 ...
 ['20101003T082539.IV.BOB..HH' '1']
 ['20101003T082539.IV.POFI..HN' '1']
 ['20101003T095135.IV.RMP..HH' '1']]


In [ ]:
# Préparation des données d'entrainement, validation et de test
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import torch 
from torch.utils.data import DataLoader

noise_matrix=events_matrix[0:10000, :, :]
events_matrix=noise_matrix[0:10000, :, :]

# Diviser les données de bruit en ensemble d'entraînement et ensemble de test
df_noise_train1, df_noise_test = train_test_split(noise_matrix, test_size=0.20, random_state=42)
df_noise_train, df_noise_validation = train_test_split(df_noise_train1, test_size=0.20, random_state=42)

trace_noise1, trace_noise_test =train_test_split(trace_names_n, test_size=0.20, random_state=42)
trace_noise_train, trace_noise_val = train_test_split(trace_noise1, test_size=0.20, random_state=42)

trace_events1, trace_events_test =train_test_split(trace_names_e, test_size=0.20, random_state=42)
trace_events_train, trace_events_val = train_test_split(trace_events1, test_size=0.20, random_state=42)


df_train1, df_test = train_test_split(events_matrix, test_size=0.20, random_state=42)
df_train, df_validation = train_test_split(df_train1, test_size=0.20, random_state=42)


# Concaténer df_noise_train avec df_train
df_train_concat = np.concatenate([df_train, df_noise_train], axis=0)
trace_train_y = np.concatenate([trace_events_train, trace_noise_train], axis=0)
trace_train = trace_train_y[:,1].astype(int)

# Concaténer df_noise_test avec df_test
df_test_concat = np.concatenate([df_test, df_noise_test], axis=0)
trace_test_y= np.concatenate([trace_events_test, trace_noise_test], axis=0)
# Convertir les étiquettes en entiers
trace_test = trace_test_y[:,1].astype(int)

#print (df_test_concat)
# Concaténer df_noise_validation avec df_validation
df_val_concat = np.concatenate([df_validation, df_noise_validation], axis=0)
trace_val_y= np.concatenate([trace_events_val, trace_noise_val], axis=0)
trace_val = trace_val_y[:,1].astype(int)
"""
# Normaliser toute la donnée
mean_t = np.mean(df_train_concat)
std_t = np.std(df_train_concat)
norm_data_t = (df_train_concat - mean_t) / std_t

#print(norm_data_t)

mean_v = np.mean(df_val_concat)
std_v = np.std(df_val_concat)
norm_data_v = (df_val_concat - mean_v) / std_v

mean_ts = np.mean(df_test_concat)
std_ts = np.std(df_test_concat)
norm_data_ts = (df_test_concat - mean_ts) / std_ts
"""
# Normaliser toute la donnée
def normalize_data(data):
    mean = np.mean(data, axis=(1, 2), keepdims=True)  # Calculer la moyenne de chaque échantillon
    std = np.std(data, axis=(1, 2), keepdims=True)    # Calculer l'écart-type de chaque échantillon
    norm_data = (data - mean) / std                  # Normaliser chaque échantillon
    return norm_data

# Appliquer la normalisation sur les données d'entraînement, de validation et de test
norm_data_train = normalize_data(df_train_concat)
norm_data_val = normalize_data(df_val_concat)
norm_data_test = normalize_data(df_test_concat)
norm_data_t = df_train_concat 
norm_data_v = df_val_concat 
norm_data_ts = df_test_concat              

print(trace_train.shape)

print(norm_data_t.shape)

# Convert your numpy arrays to PyTorch tensors
x_train_tensor = torch.tensor(norm_data_t, dtype=torch.float32)
y_train_tensor = torch.tensor(trace_train, dtype=torch.long)  # long for CrossEntropyLoss

x_test_tensor = torch.tensor(norm_data_ts, dtype=torch.float32)
y_test_tensor = torch.tensor(trace_test, dtype=torch.long)

x_val_tensor = torch.tensor(norm_data_v, dtype=torch.float32)
y_val_tensor = torch.tensor(trace_val, dtype=torch.long)

print(x_val_tensor)
print( y_val_tensor)

y_train_tensor = y_train_tensor.unsqueeze(1)
y_val_tensor = y_val_tensor.unsqueeze(1)
y_test_tensor = y_test_tensor.unsqueeze(1)

# Create TensorDatasets
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)


# Créer les DataLoaders pour l'entraînement, la validation et le test
batch_size =64  # Vous pouvez changer cette valeur selon vos besoins


val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


(25600,)
(12800, 3, 12000)


In [1]:

########################################   Model1 CNN   ###########################



import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report

# Define the input shape
input_shape = (3, 12000)

# Create a deeper CNN model

from tensorflow.keras import models, layers

# Define the input shape
input_shape = (3, 12000)
EPOCHS=50
TIME_PERIODS=12000
n_sensors=3
n_classes=2
BATCH =64

model = models.Sequential([
    layers.Reshape((TIME_PERIODS, n_sensors), input_shape=(TIME_PERIODS, n_sensors)),
    layers.Conv1D(100, 10, activation='relu', input_shape=(TIME_PERIODS, n_sensors)),
    layers.Conv1D(100, 10, activation='relu'),
    layers.MaxPooling1D(3),
    layers.Conv1D(160, 10, activation='relu'),
    layers.Conv1D(160, 10, activation='relu'),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.summary()


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

data_v_transpo= np.transpose(norm_data_v, (0, 2, 1))
data_tr_transpo= np.transpose(norm_data_t, (0, 2, 1))



# Train the model
history = model.fit(data_tr_transpo, trace_train, epochs=EPOCHS, batch_size=BATCH, validation_data=(data_v_transpo, trace_val))

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

data_ts_transpo= np.transpose(norm_data_ts, (0, 2, 1))

# Make predictions on test data
predictions = model.predict(data_ts_transpo)
binary_predictions = (predictions > 0.5).astype(int)
predictions = model.predict(data_v_transpo)
data_ts_transpo


# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(data_ts_transpo, trace_test)
print('Test accuracy:', test_acc)

# Make predictions on test data
predictions = model.predict(data_ts_transpo)
binary_predictions = (predictions > 0.5).astype(int)

#Obtenir les prédictions sur l'ensemble de validation
predictions_v = model.predict(data_v_transpo)
binary_predictions_v = (predictions_v > 0.5).astype(int)



# Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(trace_test, predictions)
roc_auc = roc_auc_score(trace_test, predictions)

 
# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Compute confusion matrix
conf_matrix = confusion_matrix(trace_test, binary_predictions)

trace_val_y_array = trace_val_y

# Convertir les tenseurs en tableaux NumPy
trace_val_y_array_np = trace_val_y_array

# Concaténer les tableaux NumPy le long de l'axe 1
concatenated_array_v = np.concatenate((trace_val_y_array_np, binary_predictions_v), axis=1)

# Convertir les tenseurs TensorFlow en tableaux NumPy
trace_test_y_array = trace_test_y

# Convertir les tenseurs en tableaux NumPy
trace_test_y_array_np = trace_test_y_array
# Concaténer les tableaux NumPy côte à côte
concatenated_array_ts =np.concatenate((trace_test_y_array_np, binary_predictions), axis=1)  
concatenated_array = np.concatenate([concatenated_array_ts, concatenated_array_v], axis=0)
# Convertir le tableau NumPy concaténé en DataFrame pandas
result_df = pd.DataFrame(concatenated_array, columns=['trace_name', 'source_type','binary_predictions'])

# Sauvegarder le résultat dans un fichier CSV
result_df.to_csv('result.csv', index=False)

# Plot confusion matrix
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.xticks([0, 1])
plt.yticks([0, 1])
plt.tight_layout()

for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, format(conf_matrix[i, j], 'd'), horizontalalignment="center", color="white" if conf_matrix[i, j] > conf_matrix.max() / 2 else "black")

plt.show()

# Print classification report
class_report = classification_report(trace_test, binary_predictions)
print("Classification Report:")
print(class_report)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 12000, 3)          0         
                                                                 
 conv1d (Conv1D)             (None, 11991, 100)        3100      
                                                                 
 conv1d_1 (Conv1D)           (None, 11982, 100)        100100    
                                                                 
 max_pooling1d (MaxPooling1  (None, 3994, 100)         0         
 D)                                                              
                                                                 
 conv1d_2 (Conv1D)           (None, 3985, 160)         160160    
                                                                 
 conv1d_3 (Conv1D)           (None, 3976, 160)         256160    
                                                        

NameError: name 'norm_data_v' is not defined

In [ ]:

###########################   Model2 CNN  ##################################################


import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report

# Define the input shape
input_shape = (3, 12000)
EPOCHS=50
TIME_PERIODS=12000
n_sensors=3
n_classes=2
BATCH =64

# Define the input shape
input_shape = (3, 12000)

# Create a simple CNN model
model = models.Sequential([
    layers.Conv1D(32, 3, activation='relu', input_shape=input_shape),
    layers.MaxPooling1D(1),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(norm_data_t, trace_train, epochs=40, batch_size=BATCH, validation_data=(norm_data_v, trace_val))

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(norm_data_ts, trace_test)
print('Test accuracy:', test_acc)

# Make predictions on test data
predictions = model.predict(norm_data_ts)
binary_predictions = (predictions > 0.5).astype(int)

# Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(trace_test, predictions)
roc_auc = roc_auc_score(trace_test, predictions)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Compute confusion matrix
conf_matrix = confusion_matrix(trace_test, binary_predictions)

# Plot confusion matrix
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.xticks([0, 1])
plt.yticks([0, 1])
plt.tight_layout()

for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, format(conf_matrix[i, j], 'd'), horizontalalignment="center", color="white" if conf_matrix[i, j] > conf_matrix.max() / 2 else "black")

plt.show()

# Print classification report
class_report = classification_report(trace_test, binary_predictions)
print("Classification Report:")
print(class_report)